In [ ]:
import Test.QuickCheck( quickCheck, 
                        Arbitrary( arbitrary ),
                        oneof, elements, sized, (==>)  )
import Control.Monad -- defines liftM, liftM2, liftM3, used below
import Data.Char

In [ ]:
f :: [Int] -> Bool
f (x:xs) = and [b `mod` a == 0 | (a, b) <- zip (x:xs) xs]

prop_f = 
    f [1,1,-2,6,18,-18,180] == True &&
    f [17] == True &&
    f [1,1,2,3,6,18] == False &&
    f [1,2,6,3,9] == False
    
quickCheck prop_f

In [ ]:
g :: [Int] -> Bool
g [x] = True
g (x:xs) | (head xs) `mod` x == 0 = g xs
         | otherwise = False

prop_g = 
    g [1,1,-2,6,18,-18,180] == True &&
    g [17] == True &&
    g [1,1,2,3,6,18] == False &&
    g [1,2,6,3,9] == False
    
quickCheck prop_g

In [ ]:
p :: [Int] -> Int
p xs = product [x^2 | x <- xs, x < 0]

prop_p = 
    p [13] == 1 &&
    p [] == 1 &&
    p [-3,3,1,-3,2,-1] == 81 &&
    p [2,6,-3,0,3,-7,2] == 441 &&
    p [4,-2,-1,-3] == 36
    
quickCheck prop_p

In [ ]:
q :: [Int] -> Int
q [] = 1
q (x:xs) | x < 0 = x^2 * q xs
         | otherwise = q xs
         
prop_q =
    q [13] == 1 &&
    q [] == 1 &&
    q [-3,3,1,-3,2,-1] == 81 &&
    q [2,6,-3,0,3,-7,2] == 441 &&
    q [4,-2,-1,-3] == 36

quickCheck prop_q

In [ ]:
r :: [Int] -> Int
r = foldr (\x acc -> if x < 0 then acc*x^2 else acc) 1

prop_r =
    r [13] == 1 &&
    r [] == 1 &&
    r [-3,3,1,-3,2,-1] == 81 &&
    r [2,6,-3,0,3,-7,2] == 441 &&
    r [4,-2,-1,-3] == 36

quickCheck prop_r

In [ ]:
data Expr = X
          | Const Int
          | Expr :+: Expr
          | Expr :-: Expr
          | Expr :*: Expr
          | Expr :/: Expr
          | IfZero Expr Expr Expr
          deriving (Eq, Ord)

-- turns an Expr into a string approximating mathematical notation

showExpr :: Expr -> String
showExpr X          =  "X"
showExpr (Const n)  =  show n
showExpr (p :+: q)  =  "(" ++ showExpr p ++ "+" ++ showExpr q ++ ")"
showExpr (p :-: q)  =  "(" ++ showExpr p ++ "-" ++ showExpr q ++ ")"
showExpr (p :*: q)  =  "(" ++ showExpr p ++ "*" ++ showExpr q ++ ")"
showExpr (p :/: q)  =  "(" ++ showExpr p ++ "/" ++ showExpr q ++ ")"
showExpr (IfZero p q r)  = "(if " ++ showExpr p ++ "=0 then "
                                  ++ showExpr q ++ " else "
                                  ++ showExpr r ++ ")"

-- For QuickCheck

instance Show Expr where
    show  =  showExpr

instance Arbitrary Expr where
    arbitrary  =  sized expr
        where
          expr n | n <= 0     =  oneof [elements [X]]
                 | otherwise  =  oneof [ liftM Const arbitrary
                                       , liftM2 (:+:) subform2 subform2
                                       , liftM2 (:-:) subform2 subform2
                                       , liftM2 (:*:) subform2 subform2
                                       , liftM2 (:/:) subform2 subform2
                                       , liftM3 (IfZero) subform3 subform3 subform3
                                       ]
                 where
                   subform2  =  expr (n `div` 2)
                   subform3  =  expr (n `div` 3)

In [ ]:
eval :: Expr -> Int -> Int
eval X x = x
eval (Const a) _ = a
eval (e1 :+: e2) x = (eval e1 x) + (eval e2 x)
eval (e1 :-: e2) x = (eval e1 x) - (eval e2 x)
eval (e1 :*: e2) x = (eval e1 x) * (eval e2 x)
eval (e1 :/: e2) x = (eval e1 x) `div` (eval e2 x)
eval (IfZero p q r) x = if (eval p x) == 0 then (eval q x) else (eval r x)

prop_eval =
    eval (X :+: (X :*: Const 2)) 3 == 9 &&
    eval (X :/: Const 3) 7 == 2 &&
    eval (IfZero (X :-: Const 3) (X:/:X) (Const 7)) 3 == 1 &&
    eval (IfZero (X :-: Const 3) (X:/:X) (Const 7)) 4 == 7 &&
    eval (Const 15 :-: (Const 7 :/: (X :-: Const 1))) 0 == 22
    
quickCheck prop_eval

In [ ]:
protect :: Expr -> Expr
protect (e1 :+: e2) = (protect e1) :+: (protect e2) 
protect (e1 :-: e2) = (protect e1) :-: (protect e2)
protect (e1 :*: e2) = (protect e1) :*: (protect e2)
protect (e1 :/: e2) = IfZero (protect e2) (Const maxBound) ((protect e1) :/: (protect e2))
protect (IfZero p q r) = IfZero (protect p) (protect q) (protect r)
protect e = e

prop_protect = 
    eval (protect (X :+: (X :*: Const 2))) 3 == 9 &&
    eval (protect (X :/: Const 3)) 7 == 2 &&
    eval (protect (IfZero (X :-: Const 3) (X:/:X) (Const 7))) 3 == 1 &&
    eval (protect (IfZero (X :-: Const 3) (X:/:X) (Const 7))) 4 == 7 &&
    eval (protect (Const 15 :-: (Const 7 :/: (X :-: Const 1)))) 0 == 22 &&
    eval (protect (Const 15 :-: (Const 7 :/: (X :-: Const 1)))) 1 == (15-maxBound) &&
    eval (protect (X :/: (X :-: X))) 2 == maxBound
    
quickCheck prop_protect